In [1]:
import torch 
import numpy as np

In [2]:
max_sequence_length = 4
idx = 0
num_sentences = 2
decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
NEG_INFTY = -1e9

eng_sentence_length, kn_sentence_length = 2, 2

eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True


encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)

In [3]:
eng_chars_to_padding_mask, kn_chars_to_padding_mask
encoder_padding_mask

tensor([[[False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True],
         [ True,  True,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]])

In [4]:
encoder_self_attention_mask

tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+09],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+09],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]]])

2 X Longest X 64
eg: 2 X 10 X 64

att: 10 X 10  
mask: 10 X 10 

tokenser: 2 X 10 
mask: 10 X 10 

        my      name     is   PAD   PAD
my      0       0        0    -inf  -inf
name    0       0        0    -inf  -inf
is      0       0        0    -inf  -inf
PAD    -inf     -inf    -inf  -inf  -inf
PAD    -inf     -inf    -inf  -inf  -inf

tokenser: 2 X 10 
emb: 2 X 10 X 64
mask: 10 X 10 


att_mask = 0 0 1
0 0 1
0 0 1
1 1 1

In [24]:
z = torch.tensor([1, 2])
for a in z:
    print(a)

tensor(1)
tensor(2)


In [44]:
s = torch.tensor([0, 0, 1])
torch.where(s == 1, 10, 20)

m_s = s.shape[0]
mask = torch.zeros((m_s, m_s))
for i in range(3):
    if s[i] == 1:
        mask[i, :] = -1
        mask[:, i] = -1

[0, 0, 1]

In [45]:
mask

tensor([[ 0.,  0., -1.],
        [ 0.,  0., -1.],
        [-1., -1., -1.]])

In [53]:
def create_pad_mask(att):
    s = len(att.tolist())
    att_mask = torch.zeros((s, s))
    for i in range(s):
        if att[i] == 1:
            att_mask[i, :] = -1
            att_mask[:, i] = -1
    return att_mask
create_pad_mask(torch.tensor([0, 0, 1]))

tensor([[ 0.,  0., -1.],
        [ 0.,  0., -1.],
        [-1., -1., -1.]])

In [15]:
input = torch.randn((2, 10, 64))
att = torch.randn((2, 10, 10))
mask = torch.randn((10, 10))

In [10]:
att.shape, mask.shape, (att + mask).shape

(torch.Size([2, 10, 10]), torch.Size([10, 10]), torch.Size([2, 10, 10]))

In [14]:
a = torch.randn((5, 5))
m = (a == a[4])
torch.where(m, -1, 0)


tensor([[ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [-1, -1, -1, -1, -1]])

In [ ]:
class Transformer(nn.Module):
    def __init__(self,
               d_model: int,
               ffn_hidden: int,
               n_head: int,
               drop_prob: int,
               n_layers: int,
               out_vocab_size: int):
            super().__init__()
            self.encoder = Encoder(d_model=d_model,
                                ffn_hidden=ffn_hidden,
                                n_head=n_head,
                                drop_prob=drop_prob,
                                n_layers=n_layers)
            self.decoder = Decoder(d_model=d_model,
                                ffn_hidden=ffn_hidden,
                                n_head=n_head,
                                drop_prob=drop_prob,
                                n_layers=n_layers)
            self.l = nn.Linear(d_model, out_vocab_size)

    def forward(self,
                enc_b,
                dec_b,
                enc_mask,
                dec_mask):
          enc_out = self.encoder(enc_b, enc_mask)
          dec_out = self.decoder(dec_b, dec_mask, enc_out)
          out = self.l(dec_out)
          return out
    
trans = Transformer(d_model=d_model,
            ffn_hidden=ffn_hidden,
            n_head=n_head,
            drop_prob=drop_prob,
            n_layers=1,
            out_vocab_size=10)
trans(x, 
      x, 
      None,
      None)

In [65]:
L = 10
mask = torch.tril(torch.ones((L, L)))
mask[mask == 0] = -torch.inf
mask[mask == 1] = 0
mask

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [63]:
mask + mask

tensor([[2., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [2., 2., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [2., 2., 2., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [2., 2., 2., 2., -inf, -inf, -inf, -inf, -inf, -inf],
        [2., 2., 2., 2., 2., -inf, -inf, -inf, -inf, -inf],
        [2., 2., 2., 2., 2., 2., -inf, -inf, -inf, -inf],
        [2., 2., 2., 2., 2., 2., 2., -inf, -inf, -inf],
        [2., 2., 2., 2., 2., 2., 2., 2., -inf, -inf],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., -inf],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]])